In [ ]:
import awe.utils
awe.utils.init_notebook()

In [ ]:
import awe.data.set.pages
import awe.training.versioning
import awe.training.params
import awe.training.trainer
awe.utils.reload('awe', exclude=['awe.data.glove'])

In [ ]:
params = awe.training.params.Params.load_user(normalize=True)
params

In [ ]:
latest_version = awe.training.versioning.Version.get_latest()
latest_params = awe.training.params.Params.load_version(latest_version)
params.difference(latest_params)

In [ ]:
trainer = awe.training.trainer.Trainer(params, prev_trainer=globals().get('trainer'))

In [ ]:
trainer.load_pretrained()

In [ ]:
trainer.load_dataset()

In [ ]:
trainer.ds.clear_cache(awe.data.set.pages.ClearCacheRequest(
    dom=False,
    labels=False,
    #dom_dirty_flags=True,
))

In [ ]:
trainer.init_features()

In [ ]:
trainer.split_data()

In [ ]:
trainer.create_dataloaders()

In [ ]:
trainer.explore_data().iloc[:5]

In [ ]:
trainer.create_model()
trainer.model

In [ ]:
trainer.create_version()

In [ ]:
trainer.train()

In [ ]:
trainer.validate(trainer.create_run(trainer.val_pages, desc='val'))

In [ ]:
pred_pages = trainer.val_pages[10:20]
[p.html_path for p in pred_pages]

In [ ]:
preds = trainer.predict(trainer.create_run(pred_pages, desc='pred'))
trainer.decode(preds)